In [1]:
%pip install pandas
%pip install streamlit
%pip install matplotlib


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.0 MB 7.1 MB/s eta 0:00:02
   ---------- ----

In [2]:
import pandas as pd
import datetime

In [3]:
#import data from csv file
path=pd.read_csv(r"C:\Users\balam\Downloads\traffic_stops.csv",low_memory=False)

In [4]:
df=pd.DataFrame(path)

In [5]:
#remove the missing value
df.dropna(axis=1,how='all',inplace=True)

In [6]:
#remove unwanted colum
df.drop('driver_age_raw',axis=1,inplace=True)
df.drop('violation_raw',axis=1,inplace=True)

In [7]:
df.fillna({
    'search_type':'none'
},inplace=True)

In [8]:
#Replace nan value
df = df.where(pd.notnull(df), None)

In [9]:
#import mysql
%pip install mysql-connector-python

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.4 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/16.4 MB 6.5 MB/s eta 0:00:03
   -------- ------------------------------- 3.4/16.4 MB 6.5 MB/s eta 0:00:02
   ------------ --------------------------- 5.2/16.4 MB 6.9 MB/s eta 0:00:02
   ---------------- ----------------------- 6.8/16.4 MB 7.2 MB/s eta 0:00:02
   ------------------ --------------------- 7.6/16.4 MB 6.5 MB/s eta 0:00:02
   --------------------- ------------------ 8.7/16.4 MB 6.1 MB/s eta 0:00:02
   ---------------------- ----------------- 9.2/16.4 MB 5.9 MB/s eta 0:00:02
   ------------------------ --------------- 10.0/16.4 MB 5.5 MB/s eta 0:00:02
   -------------------------- ------------- 10.7/16.4 MB 5.2 MB/s eta 0:00:02
   ---------------------------- ----------- 11.5/16.4 MB 5.1 MB/s eta 0:00:01
   ------------------------------ --------- 12.3/16.4 MB 5.0 MB/s eta 0:00:01
  

In [10]:
#sql connection
import mysql.connector
mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

In [12]:
#create new database
mycursor.execute("CREATE DATABASE secure")

In [13]:
#use database
mycursor.execute('USE secure')
mydb.commit()

In [14]:
#Table creation

mycursor.execute("""
CREATE TABLE IF NOT EXISTS `check` (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(50),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(50),
    drug_related_stop_boolean BOOLEAN,
    vehicle_number VARCHAR(50)
)
""")

In [15]:
#insert query
# Replace NaN with None to avoid SQL errors
import numpy as np
df = df.replace({np.nan: None})

for index, row in df.iterrows():
    mycursor.execute("""
        INSERT INTO `check` (
            stop_date, stop_time, country_name, driver_gender, driver_age, driver_race, violation,
            search_conducted, search_type, stop_outcome, is_arrested, stop_duration,
            drug_related_stop_boolean, vehicle_number
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['stop_date'], row['stop_time'], row['country_name'], row['driver_gender'], row['driver_age'],
        row['driver_race'], row['violation'], row['search_conducted'], row['search_type'], row['stop_outcome'],
        row['is_arrested'], row['stop_duration'], row['drugs_related_stop'], row['vehicle_number']
    ))

# Commit only **once** after the loop
mydb.commit()

In [16]:
df.isnull().sum()

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age            0
driver_race           0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64

In [17]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           65538 non-null  object
 1   stop_time           65538 non-null  object
 2   country_name        65538 non-null  object
 3   driver_gender       65538 non-null  object
 4   driver_age          65538 non-null  int64 
 5   driver_race         65538 non-null  object
 6   violation           65538 non-null  object
 7   search_conducted    65538 non-null  bool  
 8   search_type         65538 non-null  object
 9   stop_outcome        65538 non-null  object
 10  is_arrested         65538 non-null  bool  
 11  stop_duration       65538 non-null  object
 12  drugs_related_stop  65538 non-null  bool  
 13  vehicle_number      65538 non-null  object
dtypes: bool(3), int64(1), object(10)
memory usage: 5.7+ MB


None